# Confidence


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# Set TensorFlow logging to DEBUG for detailed logs (optional)
tf.get_logger().setLevel('DEBUG')

# Paths to the model and data
model_save_path = '../FINALMODEL_cnn_classifier.hdf5'
tflite_save_path = '../FINALMODEL_cnn_classifier.tflite'
dataset_path = '../datasets/keypoint_for_testing.csv'
refinement_csv_path = '../datasets/refinement_for_model_1.csv'

# Load the best Keras model
model = load_model(model_save_path)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Function to provide data for TFLite model quantization
def representative_dataset_gen():
    for _ in range(100):
        data = np.random.rand(1, 42).astype(np.float32)
        yield [data]

# Convert the model to TensorFlow Lite with quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_gen
try:
    tflite_model = converter.convert()
    with open(tflite_save_path, 'wb') as f:
        f.write(tflite_model)
    print("Quantized model saved to", tflite_save_path)
except Exception as e:
    print("Failed to convert with manual quantization:", e)

# Load the TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path=tflite_save_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Load testing dataset
X_test = np.loadtxt(dataset_path, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))
y_test = np.loadtxt(dataset_path, delimiter=',', dtype='int32', usecols=(0))

# Inference
predictions, confidence_scores = [], []
for test_sample in X_test:
    interpreter.set_tensor(input_details[0]['index'], np.array([test_sample], dtype=np.float32))
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    predictions.append(np.argmax(output_data))
    confidence_scores.append(np.max(output_data))

# Evaluation
accuracy = np.mean(np.array(predictions) == y_test)
print(f"Test Accuracy: {accuracy:.4f}")

# Confusion Matrix
cm = confusion_matrix(y_test, predictions)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=range(26), yticklabels=range(26))
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

# Accuracy over time plot
accuracies = [np.mean(predictions[:i + 1] == y_test[:i + 1]) for i in range(len(predictions))]
plt.plot(accuracies)
plt.xlabel('Number of Samples')
plt.ylabel('Accuracy')
plt.title('Accuracy Over Time')
plt.show()

# Prepare data for further refinement
low_confidence_or_incorrect = (np.array(confidence_scores) < 0.6) | (np.array(predictions) != y_test)
X_refinement = X_test[low_confidence_or_incorrect]
y_refinement = y_test[low_confidence_or_incorrect]

# Save refinement data
refinement_data = np.hstack((y_refinement.reshape(-1, 1), X_refinement))
refinement_df = pd.DataFrame(refinement_data, columns=['label'] + [f'feature_{i}' for i in range(42)])
refinement_df.to_csv(refinement_csv_path, index=False)
print(f"Refinement data saved to {refinement_csv_path}")


: 